In [1]:
import pandas as pd
import os

In [2]:
# Caminho completo do arquivo
caminho_arquivo = "./Reconciliation automation - FUNK - v2.0.xlsm"

# Extrai apenas o nome do arquivo
nome_arquivo = os.path.basename(caminho_arquivo)

print(nome_arquivo)  # Saída: planilha.xlsx

Reconciliation automation - FUNK - v2.0.xlsm


In [3]:
data = pd.read_excel(nome_arquivo, sheet_name="Overview", header=2, index_col=0)

In [4]:
display(data)

,2025-03-11 00:00:00,2025-05-11 00:00:00,2025-11-11 00:00:00,17-11-2025,17-11-2025.1
Settled,54,26,1,54,54
Pending,240,212,27,29,29
Participants,27,1002,1004,1015,1015
ISIN,1333,20217,7045,6569,6569
Holdings,2738,2769,238,230,230
Accounts,408,408,408,25944,25944


In [5]:
df_sem_todos_nan = data.dropna(axis=1, how='all')

In [6]:
display(df_sem_todos_nan)


,2025-03-11 00:00:00,2025-05-11 00:00:00,2025-11-11 00:00:00,17-11-2025,17-11-2025.1
Settled,54,26,1,54,54
Pending,240,212,27,29,29
Participants,27,1002,1004,1015,1015
ISIN,1333,20217,7045,6569,6569
Holdings,2738,2769,238,230,230
Accounts,408,408,408,25944,25944


In [7]:
df_sem_todos_nan_cp = df_sem_todos_nan

In [8]:
display(df_sem_todos_nan_cp)

,2025-03-11 00:00:00,2025-05-11 00:00:00,2025-11-11 00:00:00,17-11-2025,17-11-2025.1
Settled,54,26,1,54,54
Pending,240,212,27,29,29
Participants,27,1002,1004,1015,1015
ISIN,1333,20217,7045,6569,6569
Holdings,2738,2769,238,230,230
Accounts,408,408,408,25944,25944


In [9]:
df_sem_todos_nan_cp_numeric = df_sem_todos_nan_cp.apply(pd.to_numeric, errors="coerce")

In [10]:
display(df_sem_todos_nan_cp_numeric)

,2025-03-11 00:00:00,2025-05-11 00:00:00,2025-11-11 00:00:00,17-11-2025,17-11-2025.1
Settled,54,26,1,54,54
Pending,240,212,27,29,29
Participants,27,1002,1004,1015,1015
ISIN,1333,20217,7045,6569,6569
Holdings,2738,2769,238,230,230
Accounts,408,408,408,25944,25944


In [11]:
import pandas as pd
import warnings


def validar_snapshots(df, limite_percentual=80):
    """
    Valida as duas últimas colunas de um DataFrame.
    
    Parâmetros:
        df : pd.DataFrame
            DataFrame limpo de NaNs
        limite_percentual : int
            Percentual de igualdade que dispara warning de quase duplicado
        
    Retorna:
        status : str
            "duplicado", "quase_duplicado", "ok"
    """


    # Seleciona as duas últimas colunas
    col_penult = df.iloc[:, -2]
    col_ult = df.iloc[:, -1]
    nome_penult = df.columns[-2]
    nome_ult = df.columns[-1]


    # Converte para numérico (apenas para comparação segura)
    col_penult_num = pd.to_numeric(col_penult, errors="coerce")
    col_ult_num = pd.to_numeric(col_ult, errors="coerce")


    # Verifica igualdade linha a linha
    iguais = col_penult_num == col_ult_num
    percentual_iguais = iguais.mean() * 100


    # Detecta diferenças
    diferencas = df.loc[~iguais, [nome_penult, nome_ult]]


    # Decisão por níveis
    if percentual_iguais == 100:
        warnings.warn(
            f"⚠️ Snapshots '{nome_penult}' e '{nome_ult}' são 100% idênticos (duplicação clara).",
            UserWarning
        )
        status = "duplicado"


    elif percentual_iguais >= limite_percentual:
        warnings.warn(
            f"⚠️ Snapshots '{nome_penult}' e '{nome_ult}' são quase idênticos "
            f"({percentual_iguais:.1f}% iguais). Possível falha de extração.",
            UserWarning
        )
        status = "quase_duplicado"


    else:
        print(f"✅ Snapshots '{nome_penult}' e '{nome_ult}' têm diferenças relevantes ({percentual_iguais:.1f}% iguais).")
        status = "ok"


    # Exibe diferenças para análise rápida
    if not diferencas.empty:
        print("\n🔍 Diferenças detectadas:")
        display(diferencas)


    return status

In [12]:
print(validar_snapshots(df_sem_todos_nan_cp_numeric))

duplicado


C:\Users\pjunior\AppData\Local\Temp\ipykernel_16116\3270356593.py:44: UserWarning: ⚠️ Snapshots '17-11-2025' e '17-11-2025.1' são 100% idênticos (duplicação clara).
  warnings.warn(


In [13]:
# Copia o DataFrame para não alterar o original
df_formatado = df_sem_todos_nan_cp_numeric.copy()


# Converte colunas para string (caso venham como datetime ou object)
colunas_str = df_formatado.columns.astype(str)


# Extrai apenas a data (YYYY-MM-DD) antes de qualquer duplicação
colunas_formatadas = [c.split()[0] for c in colunas_str]


df_formatado.columns = colunas_formatadas


display(df_formatado)


,2025-03-11,2025-05-11,2025-11-11,17-11-2025,17-11-2025.1
Settled,54,26,1,54,54
Pending,240,212,27,29,29
Participants,27,1002,1004,1015,1015
ISIN,1333,20217,7045,6569,6569
Holdings,2738,2769,238,230,230
Accounts,408,408,408,25944,25944


In [14]:


import pandas as pd
import warnings


def validar_e_normalizar_snapshots(df, limite_percentual=80):
    """
    Valida as duas últimas colunas de um DataFrame e normaliza nomes das colunas.
    
    Passos:
        1. Valida duplicação ou quase duplicação
        2. Emite warning se necessário
        3. Remove hora das colunas
        4. Mantém unicidade das colunas
        5. Retorna DataFrame pronto para extrair últimos movimentos
    
    Parâmetros:
        df : pd.DataFrame
            DataFrame limpo de NaNs
        limite_percentual : int
            Percentual de igualdade que dispara warning de quase duplicado
    
    Retorna:
        df_formatado : pd.DataFrame
            DataFrame com colunas normalizadas
        status : str
            "duplicado", "quase_duplicado", "ok"
    """


    # --- Validação das duas últimas colunas ---
    col_penult = df.iloc[:, -2]
    col_ult = df.iloc[:, -1]
    nome_penult = df.columns[-2]
    nome_ult = df.columns[-1]


    # Converte para numérico para comparação segura
    col_penult_num = pd.to_numeric(col_penult, errors="coerce")
    col_ult_num = pd.to_numeric(col_ult, errors="coerce")


    # Verifica igualdade linha a linha
    iguais = col_penult_num == col_ult_num
    percentual_iguais = iguais.mean() * 100


    # Detecta diferenças
    diferencas = df.loc[~iguais, [nome_penult, nome_ult]]


    # --- Decisão por níveis ---
    if percentual_iguais == 100:
        warnings.warn(
            f"⚠️ Snapshots '{nome_penult}' e '{nome_ult}' são 100% idênticos (duplicação clara).",
            UserWarning
        )
        status = "duplicado"


    elif percentual_iguais >= limite_percentual:
        warnings.warn(
            f"⚠️ Snapshots '{nome_penult}' e '{nome_ult}' são quase idênticos "
            f"({percentual_iguais:.1f}% iguais). Possível falha de extração.",
            UserWarning
        )
        status = "quase_duplicado"


    else:
        print(f"✅ Snapshots '{nome_penult}' e '{nome_ult}' têm diferenças relevantes ({percentual_iguais:.1f}% iguais).")
        status = "ok"


    # Mostra diferenças para análise rápida
    if not diferencas.empty:
        print("\n🔍 Diferenças detectadas:")
        display(diferencas)


    # --- Normalização de colunas ---
    df_formatado = df.copy()
    colunas_str = df_formatado.columns.astype(str)


    # Remove hora (mantém apenas a parte da data)
    colunas_formatadas = [c.split()[0] for c in colunas_str]


    # Função para manter unicidade
    def tornar_unicas(colunas):
        novas = []
        contagem = {}
        for c in colunas:
            if c not in contagem:
                contagem[c] = 0
                novas.append(c)
            else:
                contagem[c] += 1
                novas.append(f"{c}.{contagem[c]}")
        return novas


    df_formatado.columns = tornar_unicas(colunas_formatadas)


    return df_formatado, status


In [15]:
df_formatado, status = validar_e_normalizar_snapshots(df_sem_todos_nan_cp_numeric, limite_percentual=80)
print("Status da validação:", status)
display(df_formatado)


Status da validação: duplicado


C:\Users\pjunior\AppData\Local\Temp\ipykernel_16116\3696096912.py:53: UserWarning: ⚠️ Snapshots '17-11-2025' e '17-11-2025.1' são 100% idênticos (duplicação clara).
  warnings.warn(


,2025-03-11,2025-05-11,2025-11-11,17-11-2025,17-11-2025.1
Settled,54,26,1,54,54
Pending,240,212,27,29,29
Participants,27,1002,1004,1015,1015
ISIN,1333,20217,7045,6569,6569
Holdings,2738,2769,238,230,230
Accounts,408,408,408,25944,25944


In [16]:
import pandas as pd




def normalizar_colunas_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Converte TODAS as colunas que parecem datas para DD-MM-YYYY (string),
    mantendo duplicatas como .1, .2 etc.
    """
    novas_colunas = []
    contagem = {}


    for col in df.columns:
        col_str = str(col).strip()


        # tenta converter para datetime
        dt = pd.to_datetime(col_str, errors="coerce", dayfirst=True)


        if pd.notna(dt):
            base = dt.strftime("%d-%m-%Y")
        else:
            base = col_str


        # mantém unicidade
        if base not in contagem:
            contagem[base] = 0
            novas_colunas.append(base)
        else:
            contagem[base] += 1
            novas_colunas.append(f"{base}.{contagem[base]}")


    df = df.copy()
    df.columns = novas_colunas
    return df






